In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv', low_memory=False, parse_dates=['Date'])  
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


## Single aggregates

In [3]:
df.groupby('Store').mean().head()

,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
Store,,,,,,
1,3.997877,3945.704883,467.646497,0.829087,0.382166,0.204883
2,3.997877,4122.991507,486.045648,0.832272,0.382166,0.177282
3,3.997877,5741.253715,620.286624,0.826964,0.382166,0.180467
4,3.997877,8021.769639,1100.057325,0.832272,0.382166,0.183652
5,3.997877,3867.110403,444.360934,0.826964,0.382166,0.182590


## Different Aggregates for Different Columns

In [4]:
df.groupby(['Store', 'DayOfWeek']).agg({'Sales': 'mean', 'Customers': 'count'})

Sales  Customers
Store DayOfWeek                        
1     1          4946.119403        134
      2          4650.918519        135
      3          4454.474074        135
      4          4094.607407        135
      5          4516.414815        135
...                      ...        ...
1115  3          5666.155556        135
      4          5396.607407        135
      5          6208.881481        135
      6          6773.388060        134
      7             0.000000        134

[7805 rows x 2 columns]

In [5]:
df2 = df.groupby(['Store', 'DayOfWeek']).agg({'Sales': ['mean', 'max', 'min'], 'Customers': 'count'})
df2.head()

Sales           Customers
                        mean   max min     count
Store DayOfWeek                                 
1     1          4946.119403  9528   0       134
      2          4650.918519  7959   0       135
      3          4454.474074  7821   0       135
      4          4094.607407  7785   0       135
      5          4516.414815  8414   0       135

In [6]:
df2.columns = ['SalesMean', 'SalesMax', 'SalesMin', 'Count']
df2.head()

SalesMean  SalesMax  SalesMin  Count
Store DayOfWeek                                        
1     1          4946.119403      9528         0    134
      2          4650.918519      7959         0    135
      3          4454.474074      7821         0    135
      4          4094.607407      7785         0    135
      5          4516.414815      8414         0    135

In [7]:
mc_uncert = lambda x: np.std(x) / np.sqrt(x.size)
df2 = df.groupby(['Store', 'DayOfWeek']).agg(
    {'Sales': [('SalesMean', 'mean'), ('SalesUncert', mc_uncert)], 'Customers': 'count'})
df2.head()

Sales             Customers
                   SalesMean SalesUncert     count
Store DayOfWeek                                   
1     1          4946.119403  139.120367       134
      2          4650.918519   94.412423       135
      3          4454.474074   97.225056       135
      4          4094.607407  129.367657       135
      5          4516.414815  113.495466       135

In [8]:
def mc_uncert2(x):
    return np.std(x) / np.sqrt(x.size)

dfg = df.groupby(['Store', 'DayOfWeek'])
dfg.agg(
    SalesMean=('Sales', 'mean'),
    SalesUncert=('Sales', mc_uncert2)
).reset_index().head()

,Store,DayOfWeek,SalesMean,SalesUncert
0,1,1,4946.119403,139.120367
1,1,2,4650.918519,94.412423
2,1,3,4454.474074,97.225056
3,1,4,4094.607407,129.367657
4,1,5,4516.414815,113.495466
